<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

In [1]:
!pip install skillsnetwork

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00


<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
import skillsnetwork
await skillsnetwork.prepare('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip', path = "./", overwrite=True)

  0%|          | 0/80045 [00:00<?, ?it/s]

Saved to '.'


In [3]:
import os
os.listdir('./concrete_data_week4')

['test', 'train', '.DS_Store', 'valid']

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [4]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_vgg16

In [5]:
num_classes = 2
image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator_vgg16 = ImageDataGenerator(preprocessing_function = preprocess_vgg16)

train_generator_vgg16 = data_generator_vgg16.flow_from_directory('concrete_data_week4/train', target_size=(image_resize, image_resize), batch_size=batch_size_training, class_mode='categorical')
validation_generator_vgg16 = data_generator_vgg16.flow_from_directory('concrete_data_week4/valid', target_size=(image_resize, image_resize), batch_size=batch_size_validation, class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [7]:
model_vgg16 = Sequential()
model_vgg16.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
model_vgg16.add(Dense(num_classes, activation='softmax'))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
model_vgg16.layers

In [9]:
model_vgg16.layers[0].layers

In [10]:
model_vgg16.layers[0].trainable = False
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
steps_per_epoch_training = len(train_generator_vgg16)
steps_per_epoch_validation = len(validation_generator_vgg16)
num_epochs = 2

In [13]:
fit_history = model_vgg16.fit(
    train_generator_vgg16,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator_vgg16,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 233s 703ms/step - loss: 0.0974 - accuracy: 0.9644 - val_loss: 0.0247 - val_accuracy: 0.9938
Epoch 2/2
301/301 [==============================] - 175s 581ms/step - loss: 0.0188 - accuracy: 0.9958 - val_loss: 0.0149 - val_accuracy: 0.9953


In [14]:
model_vgg16.save('classifier_vgg_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [16]:
from numpy import loadtxt
from keras.models import load_model
model_resnet50 = load_model('classifier_resnet_model.h5')
model_resnet50.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [17]:
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input as preprocess_resnet50

data_generator_resnet50 = ImageDataGenerator(preprocessing_function = preprocess_resnet50)

test_generator_resnet50 = data_generator_resnet50.flow_from_directory('concrete_data_week4/test', target_size=(image_resize, image_resize), shuffle=False)
test_generator_vgg16 = data_generator_vgg16.flow_from_directory('concrete_data_week4/test', target_size=(image_resize, image_resize), shuffle=False)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [18]:
performance_resnet50 = model_resnet50.evaluate(test_generator_resnet50)
performance_vgg16 = model_vgg16.evaluate(test_generator_vgg16)

16/16 [==============================] - 10s 325ms/step - loss: 0.0115 - accuracy: 0.9960


In [19]:
print("Performance of VGG16 model:")
print("- Loss:", performance_vgg16[0])
print("- Accuracy:", performance_vgg16[1])

Performance of VGG16 model:
- Loss: 0.011533571407198906
- Accuracy: 0.9959999918937683


In [20]:
print("Performance of ResNet50 model:")
print("- Loss:", performance_resnet50[0])
print("- Accuracy:", performance_resnet50[1])

Performance of ResNet50 model:
- Loss: 0.0021788817830383778
- Accuracy: 1.0


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [86]:
predictions_vgg16 = model_vgg16.predict(test_generator_vgg16)
yhat = predictions_vgg16.argmax(1)
for pred in yhat[0:5]:
    print("Negative") if pred==0 else print("Positive")

16/16 [==============================] - 2s 140ms/step
Negative
Negative
Negative
Negative
Negative


In [87]:
predictions_resnet50 = model_resnet50.predict(test_generator_resnet50)
yhat = predictions_resnet50.argmax(1)
for pred in yhat[0:5]:
    print("Negative") if pred==0 else print("Positive")

16/16 [==============================] - 2s 114ms/step
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).